# MODELO DE REGRESIÓN DE LA COLUMNA 'price' DEL DATASET 'DIAMONDS'.

## 1.-CARGA DE LBRERÍAS.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, OrdinalEncoder
from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer, SimpleImputer
import joblib

## 2.-CARGA DEL DATASET.

In [2]:
ruta = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/refs/heads/master/'
archivo = 'diamonds.csv'
url = ruta + archivo
df = pd.read_csv(url)

## 3.-COMPROBACIÓN DEL DATASET.

### 3.1.-Cabecera del DataSet.

In [3]:
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


### 3.2.-Tamaño del DataSet.

In [4]:
num_filas = df.shape[0]
num_columnas = df.shape[1]
print(f'El DataSet cargado tiene {num_filas} filas y {num_columnas} columnas.')

El DataSet cargado tiene 53940 filas y 10 columnas.


### 3.3.-Información del DataSet.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  object 
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(1), object(3)
memory usage: 4.1+ MB


### 3.4.-Descripción del DataSet.

In [6]:
df.describe(include='all')

,carat,cut,color,clarity,depth,table,price,x,y,z
count,53940.000000,53940,53940,53940,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000,53940.000000
unique,NaN,5,7,8,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Ideal,G,SI1,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,21551,11292,13065,NaN,NaN,NaN,NaN,NaN,NaN
mean,0.797940,NaN,NaN,NaN,61.749405,57.457184,3932.799722,5.731157,5.734526,3.538734
std,0.474011,NaN,NaN,NaN,1.432621,2.234491,3989.439738,1.121761,1.142135,0.705699
min,0.200000,NaN,NaN,NaN,43.000000,43.000000,326.000000,0.000000,0.000000,0.000000
25%,0.400000,NaN,NaN,NaN,61.000000,56.000000,950.000000,4.710000,4.720000,2.910000
50%,0.700000,NaN,NaN,NaN,61.800000,57.000000,2401.000000,5.700000,5.710000,3.530000
75%,1.040000,NaN,NaN,NaN,62.500000,59.000000,5324.250000,6.540000,6.540000,4.040000


### 3.4.-Filas con valores '0'.

In [7]:
filas_con_cero = df[df.isin([0, 0.0, '0']).any(axis=1)]
filas_con_cero.shape[0]-1

19

Hay 19 filas con algún valor 0 en alguna de sus columnas.

Sustituimos los valores 0 o 0.0 por la mediana.

In [8]:
for col in df.select_dtypes(include=np.number).columns:
    mediana = df[col].median()
    df.loc[df[col].isin([0, 0.0]), col] = mediana

In [9]:
filas_con_cero = df[df.isin([0, 0.0, '0']).any(axis=1)]
filas_con_cero

,carat,cut,color,clarity,depth,table,price,x,y,z


Ya no hay celdas con valores 0.

## 4.-COLUMNA A PREDECIR.

In [10]:
col_predict = 'price'

## 5.-MODELADO DE DATOS.

### 5.1.-División de tipos de columnas.

In [11]:
# Se crea una lista conteniendo los nombre de las columnas con valores numéricos.
col_num = df.select_dtypes(include=[np.number]).columns.tolist()
col_num.remove(col_predict) # Se elimina de la lista la columna a predecir.
print(col_num)

['carat', 'depth', 'table', 'x', 'y', 'z']


In [12]:
# Se crea una lista conteniendo los nombre de las columnas con valores categóricos.
col_cat = df.select_dtypes(exclude=[np.number]).columns.tolist()
print(col_cat)

['cut', 'color', 'clarity']


### 5.2.-Separación de datos de prueba y test.

In [13]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(col_predict, axis=1), df[col_predict], test_size=0.2, random_state=42)

### 5.3.-Generación de Pipelines.

#### 5.3.1.-Pipelines para variables numéricas.

In [16]:
pipeline_num = Pipeline(steps = [
    ('imputer', IterativeImputer(random_state=42)),
    ('scaler', RobustScaler())
])

#### 5.3.2.-Pipelines para variables categóricas.

En este caso, existen columnas categóricas de tipo ordinal: 'cut', 'color' y 'clarity'. Se ha de realizar el encoding apropiadamente, de acuerdo a su orden establecido. 

In [18]:
orden_cut = ['Fair', 'Good', 'Very Good', 'Premium', 'Ideal']
orden_color = ['D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
orden_clarity = ['I1', 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF', 'FL']

Se definen las listas de las columnas ordinales.

In [19]:
orden_columnas = ['cut', 'color', 'clarity']
orden_categorias = [orden_cut, orden_color, orden_clarity]

In [ ]:
pipeline_ordinal_cat = Pipeline(steps =[
    ('imputer', SimpleImputer(strategy='most_frecuent')),
    ('encoder', OrdinalEncoder(categories=))
])